# First-class functions

Functions in Python are first-class objects. Programming language theorists define a "first class object" as a program entity that can be:
 * Created at runtime;
 * assigned to a variable or element in a data structure;
 * Passed as an argument to a function;
 * returned as the result of a function.

In [1]:
def factorial(n):
    """ returns n!"""
    return 1 if n<2 else n*factorial(n-1)

In [4]:
factorial.__doc__

' returns n!'

## Higher-order functions
A function that takes a function as argument or returns a function as a result is a higher-order function. One example is `map()`.

In [5]:
list(map(factorial, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

In [6]:
fruits = ["strawberry", "fig", "apple", "cherry", "rasperry", "banana"]
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'rasperry', 'strawberry']

Sorting a list of words by their reversed spelling:

In [7]:
sorted(fruits, key=lambda word:word[::-1])

['banana', 'apple', 'fig', 'strawberry', 'cherry', 'rasperry']

## Modern replacements of map, filter and reduce

A listcomp or a genexp does the same job of map and reduce combined, but is more readable.

In [8]:
[factorial(n) for n in range(6) if n%2]

[1, 6, 120]

In [9]:
list(map(factorial, filter(lambda n: n%2, range(6))))

[1, 6, 120]

 * From Python 3.0 onwards, reduce is not a built-in.

Following shows how easy is to use `sum()` method instead of reduce() for computing the sum of elements.

In [10]:
from functools import reduce
from operator import add

reduce(add, range(100))

4950

In [11]:
sum(range(100))

4950

Other reducing built-ins are all and any.

all(iterable)
> return True if every element of the iterable is truthy;

In [12]:
all([])

True

any(iterable)
> return True if any element of the iterable is truthy;

In [13]:
any([])

False

## Anonymous functions

The lambda keyword creates an anonymous function within a Python expression.

__The body of lambda must be pure expressions__, means
 * body of lambda cannot make assignments or any other python statement such as while, try etc.
 
Outside the limited context of arguments to higher-order functions, anonymous functions are rarely useful in Python. The syntactic restrictions tend to makenon-trivial lambdas either unreadable or unworkable.

## The seven flavours of callable objects

To determine whether an object is callable, use the **callable()** built-in function.

In [14]:
callable(sum)

True

In [18]:
a = 2
callable(a)

False

The Python Data Model documentation lists seven callable types:

_User-defined functions_
>created with def or lambda expressions.

_Built-in functions_
>a function implemented in C(for CPython), like len or time.strftime.

_Built-in methods_
>methods implemented in C, like dict.get.

_Methods_
>functions defined in the body of a class.

_Classes_
>When invoked, a class runs its `__new__` method to create an instance and then `__init__` to initialize it, and finally the instance is returned to the caller. Since there is no new operator in python, calling a class is like calling a function.

_Class instances_
>if a class defines a `_call_` method, then its instances may be invoked as functions.

_Generator functions_
>functions or methods that use the yield keyword. When called, generator functions return a generator object.

## User defined callable types
Just implement a `__call__` method inside the class.

In [21]:
import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('Pick from empty BingoCage')
            
    def __call__(self):
        return self.pick()

In [22]:
bingo = BingoCage(range(3))
bingo.pick()

2

In [23]:
bingo()

1

In [24]:
callable(bingo)

True

In [105]:
print(dir(factorial))

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


`__dict__`

Like the instances of a plain user-defined class, a function uses the `__dict__` attribute to store user attributes assigned to it. 

In [26]:
def upper_case_name(obj):
    return (f"{obj.first_name} {obj.last_name}").upper()
upper_case_name.short_description = 'Customer name'

Assigning arbitrary attributes to functions is not a very common practise in general, but Django is one framework that uses it.

_Listing attributes of functions that don't exist in plain instances._

In [27]:
class C: pass
obj = C()

def func(): pass

sorted(set(dir(func))-set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

<img src='images/functions1.png'/>

## From positional to keyword-only parameters

The following tag() function generates HTML.

In [36]:
def tag(name, *content, cls=None, **attrs):
    """Generate one or more HTML tags."""
    if cls is not None:
        attrs['class'] = cls
        
    if attrs:
        attr_str = ''.join(f'{attr}={value}' 
                          for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    
    if content:
        return '\n'.join(f'<{name} {attr_str}>{c}</{name}>' for c in content)
    else:
        return f'<{name} {attr_str}/>'

In [37]:
tag('br')

'<br />'

In [38]:
tag('p', 'hello')

'<p >hello</p>'

In [39]:
tag('p', 'hello', 'world', cls='sidebar', id=123)

'<p class=sidebarid=123>hello</p>\n<p class=sidebarid=123>world</p>'

Keyword-only arguments are a new feature in Python 3. The cls parameter can only be given as a keyword argument - it will never capture unnamed positional arguments.
* To specify keyword-only arguments when defining a function, name them after the argument prefixed with *.
* If you don't want to support variable positional arguments but still want keyword-only arguments, put a * by itself in the signature.

In [40]:
def f(a, *, b):
    return a,b

In [44]:
f(1, b=2)

(1, 2)

Not that the keyword-only argument do not need to have a default value; they can be mandatory as in the example above.

## Retrieving information about parameters
Within a function object,
* `__default__` attribute holds a tuple with the default values of positional and keyword arguments.
* The defaults for keyword-only arguments appear in `__kwdefaults__`.
* The names of the arguments, are found within the `__code__` attribute, which is a reference to a code object with many attributes of its own.

There are libraries like **Bombo** which uses make use of this attributes. 

-- Check it.

In [46]:
def clip(text, max_len=80):
    """Return text clipped at the last space before or after max_len """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len) 
        if space_before >= 0:
            end = space_before 
        else:
            space_after = text.rfind(' ', max_len) 
            if space_after >= 0:
                end = space_after
    if end is None: # no spaces were found
        end = len(text) 
    return text[:end].rstrip()

In [47]:
clip.__defaults__

(80,)

In [48]:
clip.__code__.co_varnames

('text', 'max_len', 'end', 'space_before', 'space_after')

In [49]:
clip.__code__.co_argcount

2

From `clip.__code__.co_varnames`, argument names are the first `clip.__code__.co_argcount` variable names. `clip.__code__.co_argcount` does not include any variable arguments prefixed with * or **.

The default values are identified only by their position in the `__defaults__` tuple, so to link each with the respective argument you have to scan from last to first.

Fortunately there is a better way, using the **inspect** module.

In [51]:
from inspect import signature

sig = signature(clip)
sig

<Signature (text, max_len=80)>

In [52]:
sig.parameters

mappingproxy({'text': <Parameter "text">, 'max_len': <Parameter "max_len=80">})

Each parameter instance has attributes such as name, default and kind.  The special value `inspect._empty` denotes parameters with no default — which makes sense considering that None is a valid — and popular — default value.

For more, Refer: Fluent Python, Ch -5, section: Retrieving information about parameters 

In [54]:
print(sig.parameters['max_len'].default)
print(sig.parameters['max_len'].kind)

80
POSITIONAL_OR_KEYWORD


An `inspect.Signature` object has a bind method that takes any number of arguments and binds them to the parameters in the signature. This can be used by frameworks to validate arguments prior to the actual function invocation.

In [78]:
sig = signature(tag)
my_tag = {'name':'img', 'title':'Sunset Boulevard', 'cls':'framed'}
bound_args = sig.bind(**my_tag)
bound_args

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard'})>

In [79]:
del my_tag['name']
try:
    bound_args = sig.bind(**my_tag)
except Exception as e:
    import sys
    print(e, file=sys.stderr)

missing a required argument: 'name'


Frameworks and tools like IDEs can use this information to validate the code.

## Function annotations

Python 3 provides syntax to attach metadata to the parameters of a function declaration and its return value.

In [81]:
def clip(text:str, max_len:'int > 0'=80)->str:
    pass

* Each argument in the function declaration may have an annotation expression preceded by :.
* If there is a default value, the annotations goes between the argument name and the = sign.
* To annotate the return value, add -> between ending ) and : of function declaration.

In [82]:
clip.__annotations__

{'text': str, 'max_len': 'int > 0', 'return': str}

The only thing Python does with annotation is to store them in the `__annotation__` attribute of the function. Means annotation have no meaning to the Python interpreter.

They are just metadata that may be used by tools such as IDEs, frameworks and decorators. 

### Extracting annotations from the function signature

In [83]:
sig = signature(clip)

In [84]:
sig.return_annotation

str

In [86]:
for param in sig.parameters.values():
    print(f'{param.annotation} : {param.name} = {param.default}')

<class 'str'> : text = <class 'inspect._empty'>
int > 0 : max_len = 80


# Packages for functional programming

### The operator module

Consider the implementation of factorial of a number:

In [87]:
def fact(n):
    return reduce(lambda a,b:a*b, range(1, n+1))

To save the trouble of writing trivial functions like lambda a,b:a*b ,the operator module provides functions equivalents for dozens of arithmetic operators.

In [88]:
from operator import mul

def fact(n):
    return reduce(mul, range(1, n+1))

Similarly, using `itemgetter` and `attrgetter`.

In [89]:
from operator import itemgetter

metro_data = [
 ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
 ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
 ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
 ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
 ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)) 
]

for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


If you pass multiple index arguments to itemgetter, the function it builds will returns the tuples with the extracted values:

In [91]:
cc_name = itemgetter(1,0)
callable(cc_name)

True

In [92]:
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


A sibling of itemgetter is `attrgetter()`, _which creates functions to extract object attributes by name_. If several attribute names are passed, it also returns a tuple of values.

In addition, if any of the argument name contains '.', attrgetter navigates through nested objects to retrieve the attribute. 

In [93]:
from collections import namedtuple

In [94]:
LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')

metro_areas = [Metropolis(name, cc, pop, LatLong(lat, longitude)) 
               for name, cc, pop, (lat, longitude) in metro_data]

In [97]:
from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat')
for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


Here is a partial list of functions defined in operator.

In [104]:
import operator
print([name for name in dir(operator) if not name.startswith('_')])

['abs', 'add', 'and_', 'attrgetter', 'concat', 'contains', 'countOf', 'delitem', 'eq', 'floordiv', 'ge', 'getitem', 'gt', 'iadd', 'iand', 'iconcat', 'ifloordiv', 'ilshift', 'imatmul', 'imod', 'imul', 'index', 'indexOf', 'inv', 'invert', 'ior', 'ipow', 'irshift', 'is_', 'is_not', 'isub', 'itemgetter', 'itruediv', 'ixor', 'le', 'length_hint', 'lshift', 'lt', 'matmul', 'methodcaller', 'mod', 'mul', 'ne', 'neg', 'not_', 'or_', 'pos', 'pow', 'rshift', 'setitem', 'sub', 'truediv', 'truth', 'xor']


##### methodcaller

It is somewhat similar to attrgetter and itemgetter. The function it creates calls a method by name on the object given as argument.

In [106]:
from operator import methodcaller

s = 'the time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [107]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)

'the-time-has-come'

### Freezing arguments with functools.partial